# Modélisation

### 0 - importation des modules 

In [137]:
# Module pandas pour manipuler le dataframe ensuite constitué :
import pandas as pd

# Module numpy pour manipuler essentiellement des arrays :
import numpy as np
import matplotlib.pyplot as plt

# Module scikitlearn pour la modélisation : 
import sklearn

On importe notre propre module ***preprocessing_data*** qui regroupe les fonctions qui ont servi pour les étapes de preprocessing :

In [138]:
import sys
sys.path.append('./modules')
from modelisation import preprocessing_data

In [139]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1 - Sélection du modèle

Avant de se lancer dans les étapes de machine learning, il est important de bien comprendre ce que l'on souhaite prédire, et avec quelles variables, quelles sont les types de toutes ces variables, afin de comprendre quel sera le modèle le plus adapté à notre jeu de données.

On peut commencer par importer à nouveau notre jeu de données :

In [140]:
df = pd.read_csv("data/data_cleaned.csv", sep = "\t")

In [141]:
len(df)

188255

In [142]:
df.head(3)

,tconst,title,year,runtime,genres,rate,votes,director,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H..."
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ..."
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ..."


Nous souhaitons prédire la note (spectateurs) d'un film : ***rate***. Il s'agit d'une variable quantitative, continue sur le segment [1,10].
Pour prédire cette note, nous allons utiliser différentes variables : 
- ***year*** : l'année de sortie du film (quantitative discrète)
- ***runtime*** : la durée du film (quantitative discrète)
- ***votes*** : le nombre de votes recensés pour former la note du film (quantitative discrète)
- ***genres*** : le genre du film (qualitative)
- ***director*** : le réalisateur du film (qualitative)
- ***casting*** : les acteurs du film (liste de qualitatives)

Critère 1 : Quantité de données ;
Critère 2 : structure des données (ici données structurées) ;
Critère 3 : Variables qualitatives / quantitatives

Une regression linéaire semble tout à fait adaptée pour démarrer.

### 2 - Preprocessing

Avant de mettre en oeuvre une modélisation via un modèle de regression linéaire, certaines variables doivent être transformées.

Plus précisément, nous avons besoin de transformer les variables qualitatives **genres**, **casting**, et **director** en variables quantitatives.

Pour la variable **genres**, comme vu précédemment il y a seulement 28 genres présents dans la base de données, donc 28 modalités prises par la variable **genres**. Un endodage du type "***One Hot Encoding***" fera tout à fait l'affaire.

(expliquer le principe du One Hot Encoding)

#### 2.1 - Transformation de la variable ***genres***

(voir la video du mec pour voir comment bien faire)

#### 2.2 - Transformation de la variable ***director*** 

Nous voyons que la situation pour ces deux variables n'est pas la même que la pour la variable ***genres***. En effet un encodage du type "***One Hot Encoding***" risque d'être totalement inapproprié à la structure de ces deux variables. Plus préciséments, contrairement aux genres, les nombres de réalisateurs et acteurs sont très conséquents.

Par exemple pour les directeurs, combien y en a-t-il?

In [143]:
df["director"].nunique()

68373

Il y a 68 373 réalisateurs différents dans notre base de données. Le One Hot Encoding va ainsi générer une sparse matrix de taille 68373x68373 ! Ce n'est pas adapté.

Nous avons ainsi trouvé une autre alternative : à partir d'une formule mathématique, nous allons affecter un score aux réalisateurs, basé sur les notes qu'ont reçues leurs films et le nombre de films qu'ils ont produits.

La formule est la suivante : 

$$\frac{1}{rank}rate^2$$

Avec :

- ***rank*** : le classement du film dans la base de données (selon sa note)
- ***rate*** : la note du film par les telespectateurs (la variable **rate** de la base de données)

Ainsi pour un réalisateur donné, on obtient son score en faisant la somme de tous les scores que ses films lui ont fait obtenir.

Pour appliquer cette formule il faut bien evidemment calculer la variable rank pour chaque film. 

In [144]:
df = preprocessing_data.add_ranks_films(df)

On vérifie que le rank est bien apparu dans la base, c'est le cas, par exemple on voit que le film "Rebecca" est classée 98 476ème (c'est à dire 98 476ème meilleure note) :

In [145]:
df.head(5)

,tconst,title,year,runtime,genres,rate,votes,director,casting,index,rank
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",0,33273.0
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ...",1,6429.0
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ...",2,98476.0
3,tt10682266,Hubie Halloween,2020,102,"Comedy, Fantasy, Mystery",5.2,27526,Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',...",3,140355.0
4,tt2222042,Love and Monsters,2020,109,"Action, Adventure, Comedy",7.1,10780,Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ...",4,28509.0


Maintenant qu'on a le rank, on peut calculer le score de chaque réalisateur. C'est ce qui a été implémenté dans la fonction ***add_score_realisateurs*** :

In [146]:
df = preprocessing_data.add_scores_realisateurs(df)

On peut vérifier que la fonction a bien marché sur la base de données, on retrouve le score_directeur :

In [147]:
df.head(5)

,tconst,title,year,runtime,genres,rate,votes,director,casting,index,rank,score_film,score_realisateur
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H...",0,33273.0,0.001473,0.001473
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ...",1,6429.0,0.009708,0.012974
2,tt4209788,Le grand jeu,2017,140,"Biography, Crime, Drama",7.4,136590,Aaron Sorkin,"['Jessica Chastain', 'Idris Elba', 'Kevin Cost...",276,16762.0,0.003267,0.012974
3,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",6.0,11829,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ...",2,98476.0,0.000366,0.004158
4,tt0462335,High-Rise,2015,119,"Drama, Sci-Fi",5.6,37449,Ben Wheatley,"['Tom Hiddleston', 'Jeremy Irons', 'Sienna Mil...",2204,121716.0,0.000258,0.004158


#### 2.2 - Transformation de la variable ***casting*** 

Le principe va être le même que pour la variable ***director***, la seule grande différence est que nous avons plusieurs acteurs pour un même film en grande majorité. 

Le principe va être le suivant :
- implémenter une fonction ***actors_list_split*** qui va permettre d'obtenir pour chaque ligne du dataframe un film et un seul nom d'acteur. Par exemple un film avec trois acteurs va produire trois lignes avec le même nom de film et un acteur différent pour chaque ligne. Le principe est le même que pour les genres lors de la visualisation.
- appliquer à nouveau une formule pour calculer le score de chaque acteur suivant le **rank** du film qui a déjà été défini

Pour permettre la première étape de split, on doit nettoyer un peu la colonne **casting** :

In [148]:
df["casting"] = df["casting"].apply(lambda x : x.replace("[","").replace("]","").replace("'","")) 

Ainsi on a juste la liste des différents acteurs, séparés par des virgules :

In [149]:
df.head(3)

,tconst,title,year,runtime,genres,rate,votes,director,casting,index,rank,score_film,score_realisateur
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,"Sacha Baron Cohen, Maria Bakalova, Tom Hanks, ...",0,33273.0,0.001473,0.001473
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,"Eddie Redmayne, Alex Sharp, Sacha Baron Cohen,...",1,6429.0,0.009708,0.012974
2,tt4209788,Le grand jeu,2017,140,"Biography, Crime, Drama",7.4,136590,Aaron Sorkin,"Jessica Chastain, Idris Elba, Kevin Costner, M...",276,16762.0,0.003267,0.012974


On applique la fonction ***split_elem_col*** implémentée dans le module ***visualisation*** :

In [150]:
import sys
sys.path.append('./modules')
from visualisation import split_method

In [151]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Tout comme le genre, comme le nombre de lignes à split est très long, on montre que la fonction fonctionne correctement uniquement sur les 100 premières lignes :

In [134]:
df = split_method.split_elem_col(df[0:99],'casting')

In [135]:
df

,tconst,title,year,runtime,genres,rate,votes,director,index,rank,score_film,score_realisateur,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,0,33273.0,0.001473,0.001473,Sacha Baron Cohen
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,0,33273.0,0.001473,0.001473,Maria Bakalova
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,0,33273.0,0.001473,0.001473,Tom Hanks
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,0,33273.0,0.001473,0.001473,Dani Popescu
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,1,6429.0,0.009708,0.012974,Eddie Redmayne
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,tt5592248,Les proies,2017,93,"Drama, Thriller",6.3,50868,Sofia Coppola,1815,78137.0,0.000508,0.011289,Colin Farrell
98,tt2132285,The Bling Ring,2013,90,"Biography, Crime, Drama",5.6,82535,Sofia Coppola,2694,121672.0,0.000258,0.011289,Katie Chang
98,tt2132285,The Bling Ring,2013,90,"Biography, Crime, Drama",5.6,82535,Sofia Coppola,2694,121672.0,0.000258,0.011289,Israel Broussard
98,tt2132285,The Bling Ring,2013,90,"Biography, Crime, Drama",5.6,82535,Sofia Coppola,2694,121672.0,0.000258,0.011289,Emma Watson


On fait l'opération pour la base complète, on obtient la base suivante :

In [166]:
df = pd.read_pickle("./data/data_actors_splitted.pkl")

In [167]:
df

,tconst,title,year,runtime,genres,rate,votes,director,rank,score_film,score_realisateur,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Sacha Baron Cohen
1,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Maria Bakalova
2,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Tom Hanks
3,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Dani Popescu
4,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",7.9,37295,Aaron Sorkin,6429.0,0.009708,0.012974,Eddie Redmayne
...,...,...,...,...,...,...,...,...,...,...,...,...
761560,tt0048284,Zemlya,1954,82,Drama,5.9,130,Amvrosi Buchma,106877.0,0.000326,0.000326,Pavel Grubnik
761561,tt4084038,Shanthi Appuram Nithya,2011,96,Drama,6.9,260,Murali Vishwa,40873.0,0.001165,0.001165,Archana Sharma
761562,tt13330846,Patra Vaitha Nerupondru,2020,134,"Action, Crime, Thriller",8.8,160,Vinoth Rajendran,598.0,0.129498,0.129498,Ranjith
761563,tt13330846,Patra Vaitha Nerupondru,2020,134,"Action, Crime, Thriller",8.8,160,Vinoth Rajendran,598.0,0.129498,0.129498,Dinesh Sadasivam


On calcule maintenant le score de chaque acteur :

In [168]:
df = preprocessing_data.add_scores_acteurs(df)

In [169]:
df

,tconst,title,year,runtime,genres,rate,votes,director,rank,score_film,score_realisateur,casting,score_acteur
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,7.0,50191,Jason Woliner,33273.0,0.001473,0.001473,Sacha Baron Cohen,0.005991
1,tt0443453,Borat: Leçons culturelles sur l'Amérique pour ...,2006,84,Comedy,7.3,361442,Larry Charles,19925.0,0.002675,0.003958,Sacha Baron Cohen,0.005991
2,tt0889583,Brüno,2009,81,Comedy,5.8,141221,Larry Charles,110166.0,0.000305,0.003958,Sacha Baron Cohen,0.005991
3,tt1645170,The Dictator,2012,83,Comedy,6.4,271848,Larry Charles,70980.0,0.000577,0.003958,Sacha Baron Cohen,0.005991
4,tt3381008,Grimsby - Agent trop spécial,2016,83,"Action, Adventure, Comedy",6.2,91074,Louis Leterrier,84793.0,0.000453,0.007325,Sacha Baron Cohen,0.005991
...,...,...,...,...,...,...,...,...,...,...,...,...,...
761560,tt0048284,Zemlya,1954,82,Drama,5.9,130,Amvrosi Buchma,106877.0,0.000326,0.000326,Sergey Feshchenko,0.000326
761561,tt0048284,Zemlya,1954,82,Drama,5.9,130,Amvrosi Buchma,106877.0,0.000326,0.000326,Pavel Grubnik,0.000326
761562,tt4084038,Shanthi Appuram Nithya,2011,96,Drama,6.9,260,Murali Vishwa,40873.0,0.001165,0.001165,Archana Sharma,0.001165
761563,tt13330846,Patra Vaitha Nerupondru,2020,134,"Action, Crime, Thriller",8.8,160,Vinoth Rajendran,598.0,0.129498,0.129498,Ranjith,0.129498


Ok c'est nickel